## Packages

In [1]:
import sqlite3

In [2]:
# importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data

In [3]:
path_bm = 'c:\\datasets\\kaggle\\BigMartSales\\datasets_12276_16823_Train_UWu5bXk.csv'
BigMart = pd.read_csv(path_bm)

In [6]:
BigMart.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


## Data Preparation

In [7]:
BigMart.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Item_Identifier,8523,1559,FDW13,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_Weight,7060,NaN,NaN,NaN,12.8576,4.64346,4.555,8.77375,12.6,16.85,21.35
Item_Fat_Content,8523,5,Low Fat,5089,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_Visibility,8523,NaN,NaN,NaN,0.066132,0.0515978,0,0.0269895,0.0539309,0.0945853,0.328391
Item_Type,8523,16,Fruits and Vegetables,1232,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_MRP,8523,NaN,NaN,NaN,140.993,62.2751,31.29,93.8265,143.013,185.644,266.888
Outlet_Identifier,8523,10,OUT027,935,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Outlet_Establishment_Year,8523,NaN,NaN,NaN,1997.83,8.37176,1985,1987,1999,2004,2009
Outlet_Size,6113,3,Medium,2793,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Outlet_Location_Type,8523,3,Tier 3,3350,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
missing_val_count_by_column = (BigMart.isnull().sum())
print("overall ", len(missing_val_count_by_column[missing_val_count_by_column > 0]) , 
      "variables with missing values" , "\nin details:","\n" , 
      missing_val_count_by_column[missing_val_count_by_column > 0])

overall  2 variables with missing values 
in details: 
 Item_Weight    1463
Outlet_Size    2410
dtype: int64


In [12]:
# imputing missing values in Item_Weight by medians by Item_Type and Outlet_Size with 'Unknown'
BigMart['Outlet_Size']=BigMart['Outlet_Size'].fillna('Unknown')

In [11]:
tofill = BigMart[BigMart.Item_Weight.notnull()].groupby('Item_Type')[['Item_Weight']].median().reset_index()
tofill

,Item_Type,Item_Weight
0,Baking Goods,11.650
1,Breads,10.600
2,Breakfast,10.695
3,Canned,12.150
4,Dairy,13.350
5,Frozen Foods,12.850
6,Fruits and Vegetables,13.100
7,Hard Drinks,10.100
8,Health and Hygiene,12.150
9,Household,13.150


## sqlite3

In [13]:
#Make the db in memory
conn = sqlite3.connect(':memory:')

In [16]:
#write the two tables
BigMart.to_sql('BigMart', conn, index=False)
tofill.to_sql('tofill', conn, index=False)
qry = '''
    select  
        BigMart.*,tofill.Item_Weight as Item_Weight_fill
    from
        BigMart left join tofill on
        BigMart.Item_Type = tofill.Item_Type
    '''
BigMart = pd.read_sql_query(qry, conn)
BigMart.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Weight_fill
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,13.35
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,11.80
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,12.35
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Unknown,Tier 3,Grocery Store,732.3800,13.10
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,13.15


so far this was something easy to do with pandas too: 
#### temporary column with the medians
BigMart = BigMart.merge(tofill, how='inner',left_on='Item_Type', right_on='Item_Type',
                         suffixes=('', '_right'))

In [19]:
# fill nas with median by Item_Type
BigMart['Item_Weight'] = BigMart.apply(lambda row: row['Item_Weight_fill'] if 
                                       np.isnan(row['Item_Weight']) else row['Item_Weight'],axis=1)
BigMart.drop(['Item_Weight_fill'], axis=1)
BigMart.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Weight_fill
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,13.35
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,11.80
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,12.35
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Unknown,Tier 3,Grocery Store,732.3800,13.10
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,13.15


In [20]:
missing_val_count_by_column = (BigMart.isnull().sum())
print("overall ", len(missing_val_count_by_column[missing_val_count_by_column > 0]) , 
      "variables with missing values" , "\nin details:","\n" , 
      missing_val_count_by_column[missing_val_count_by_column > 0])

overall  0 variables with missing values 
in details: 
 Series([], dtype: int64)


### decades using between

In [21]:
BigMart['Outlet_Establishment_Year'].unique()

array([1999, 2009, 1998, 1987, 1985, 2002, 2007, 1997, 2004], dtype=int64)

In [23]:
d = {'start': [1981,1991,2001], 'end': [1990,2000,2010],'decade': ['eighties','nineties','2000s']}
decades = pd.DataFrame(data=d)
decades

,start,end,decade
0,1981,1990,eighties
1,1991,2000,nineties
2,2001,2010,2000s


In [24]:
#write the two tables
#BigMart.to_sql('BigMart', conn, index=False)
decades.to_sql('decades', conn, index=False)
qry = '''
    select  
        BigMart.*,decades.decade as Decade_Of_Est
    from
        BigMart left join decades on
        BigMart.Outlet_Establishment_Year between decades.start and decades.end
    '''
BigMart2 = pd.read_sql_query(qry, conn)
BigMart2.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Decade_Of_Est
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,nineties
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,2000s
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,nineties
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Unknown,Tier 3,Grocery Store,732.3800,nineties
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,eighties


so this is something which would be harder to make with Pandas